In [1]:
import csv
import nltk
import pickle
import re
import sys
import string
import warnings
import random
import numpy as np
import pandas as pd

In [2]:
from sklearn.pipeline import Pipeline
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.tokenize import SyllableTokenizer
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer

In [3]:
np.random.seed(52023)

stopwords = list(stopwords.words('english'))

# Data - Search Queries 

In [4]:
Queries = pd.read_pickle('../data/AllQueries4746.p')
Queries

,query,class
0,US civil war causes,0
1,scooter brands,0
2,scooter brands reliable,0
3,scooter,0
4,scooter cheap,0
...,...,...
4741,House of dreams,1
4742,When did Desmond doss get married,1
4743,H,1
4744,find fact about dog,1


# Search Query Splits

We split search in 90/10 distribution, 90% for experiment and 10% for pipeline testing 

In [7]:
word_concreteness = pickle.load( open( "../data/concreteness/word_concreteness.p", "rb" ) )
word_concreteness['word']=word_concreteness['word'].str.lower() 
word_concreteness

,word,concreteness,label
0,roadsweeper,4.85,concrete
1,traindriver,4.54,concrete
2,tush,4.45,concrete
3,hairdress,3.93,concrete
4,pharmaceutics,3.77,concrete
...,...,...,...
39949,unenvied,1.21,abstract
39950,agnostically,1.20,abstract
39951,conceptualistic,1.18,abstract
39952,conventionalism,1.18,abstract


In [6]:
fn_allFeat = Queries[['query','class']]
fn_allQueries = Queries['query'].tolist()

In [10]:
results = absConcFeat(fn_allQueries)
results

100%|██████████| 4746/4746 [00:05<00:00, 938.58it/s] 


,query,class,ratioAbs,ratioConc
0,US civil war causes,0,0.250000,0.250000
1,scooter brands,0,0.000000,0.500000
2,scooter brands reliable,0,0.333333,0.333333
3,scooter,0,0.000000,1.000000
4,scooter cheap,0,0.500000,0.500000
...,...,...,...,...
4741,House of dreams,1,0.333333,0.000000
4742,When did Desmond doss get married,1,0.500000,0.000000
4743,H,1,0.000000,0.000000
4744,find fact about dog,1,0.750000,0.250000


## Try pipeline

Note: all function take a list of *queries* as input

In [ ]:
#--S

In [25]:
def div(n):
    a = n/2
    return a

In [37]:
div_pip = Pipeline(steps=[
    ('transform', FunctionTransformer(div)),
    
])
div_pip

Pipeline(steps=[('transform',
                 FunctionTransformer(func=<function div at 0x7fd30266dca0>))])

In [36]:
nu = div_pip.transform(4)
nu

2.0

In [40]:
nn = div_pip.fit(4)
nn

Pipeline(steps=[('transform',
                 FunctionTransformer(func=<function div at 0x7fd30266dca0>))])

In [ ]:
abs_conc_pipe = steps=[
    ('add', FunctionTransformer(add))
]

In [15]:
def add(n1, n2):
    ans = n1+n2
    return(ans)

In [16]:
add(2,3)

5

In [45]:
# try multiple functions

In [46]:
def calculation(a, b):
    def add_numbers():
        return a + b
    
    def multiply_numbers():
        return a * b
    
    addition_result = add_numbers()
    multiplication_result = multiply_numbers()
    
    return addition_result, multiplication_result

result = calculation(5, 3)
print("Addition result:", result[0])
print("Multiplication result:", result[1])

Addition result: 8
Multiplication result: 15


In [47]:
result

(8, 15)

In [48]:

data = {'Number1': [10, 20, 30],
        'Number2': [5, 8, 12]}
df = pd.DataFrame(data)

df['Addition'] = df['Number1'] + df['Number2']
df['Subtraction'] = df['Number1'] - df['Number2']

print(df)

   Number1  Number2  Addition  Subtraction
0       10        5        15            5
1       20        8        28           12
2       30       12        42           18


In [41]:
#---E

In [9]:
def absConcFeat(queries):
    word_concreteness['word']=word_concreteness['word'].str.lower() 
    aw = word_concreteness[word_concreteness['label']=='abstract']
    cw = word_concreteness[word_concreteness['label']=='concrete']

    abW = []
    coW = []
    for w_a in aw['word']:
        abW.append(w_a)
    for w_c in cw['word']:
        coW.append(w_c)

    absrtCount = []
    concCount = []
    with tqdm(total=len(fn_allQueries)) as pbar:
        for query in fn_allQueries:

            a_countWord = 0
            c_countWord = 0
            wordCount = 0
            for word in query.split(' '):
                word.lower()
                wordCount +=1
                if word in abW:
                    a_countWord +=1
                if word in coW:
                    c_countWord +=1

            absrtCount.append(a_countWord/wordCount)
            concCount.append(c_countWord/wordCount)
            pbar.update()

    fn_allFeat['ratioAbs'] = absrtCount
    fn_allFeat['ratioConc'] = concCount

    return fn_allFeat

In [51]:
abs_conc_pipe = Pipeline(steps=[
    ('transform', FunctionTransformer(absConcFeat)),
    
])
abs_conc_pipe

Pipeline(steps=[('transform',
                 FunctionTransformer(func=<function absConcFeat at 0x7fd2a80c7430>))])

In [53]:
abs_conc_pipe.transform(fn_allQueries)

100%|██████████| 4746/4746 [00:04<00:00, 955.55it/s] 


,query,class,ratioAbs,ratioConc
0,US civil war causes,0,0.250000,0.250000
1,scooter brands,0,0.000000,0.500000
2,scooter brands reliable,0,0.333333,0.333333
3,scooter,0,0.000000,1.000000
4,scooter cheap,0,0.500000,0.500000
...,...,...,...,...
4741,House of dreams,1,0.333333,0.000000
4742,When did Desmond doss get married,1,0.500000,0.000000
4743,H,1,0.000000,0.000000
4744,find fact about dog,1,0.750000,0.250000


In [ ]:
class featExtr(BaseEstimator, TransformerMixin):
    def __init__(self):
        
    

In [1]:
print("hello words")

hello words


In [4]:
np.array(12)

array(12)